In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import hyperopt
import sys
sys.path.append('../spock/')
#from simsetup import get_sim
#from modelfitting import ROC_curve, stable_unstable_hist, calibration_plot, unstable_error_fraction
try:
    plt.style.use('paper')
except:
    pass
%matplotlib inline

In [4]:
dataset = pd.read_csv('../secondMMRintData/fullFeature2MMRs.csv')

In [5]:
dataset.head()

,Unnamed: 0,index,EMcrossnear,EMfracstdnear,EPstdnear,MMRstrengthnear,twoMMRstrengthnear,EMcrossfar,EMfracstdfar,EPstdfar,MMRstrengthfar,twoMMRstrengthfar,MEGNO,MEGNOstd,prelimStable,Stable
0,0,0,0.221502,0.017511,0.002935,0.010371,0.010371,0.300893,0.085965,0.011956,0.000000,0.010371,1.942393,0.032212,True,False
1,1,1,0.190647,0.073477,0.011405,0.013957,0.013957,0.403777,0.005775,0.000461,0.000000,0.013957,22.918673,7.372319,True,False
2,2,2,0.125660,0.034270,0.001506,0.088234,0.088234,0.720557,0.016427,0.008207,0.000000,0.088234,11.884854,3.404385,True,False
3,3,3,0.216684,0.008561,0.002451,0.000000,0.000000,0.235460,0.003955,0.000146,0.005636,0.000000,2.000019,0.001839,True,True
4,4,4,0.024455,NaN,NaN,NaN,NaN,0.034058,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [6]:
def hasnull(row):
    numnulls = row.isnull().sum()
    if numnulls == 0:
        return 0
    else:
        return 1

def tmax(row):
    #sim = get_sim(row, csvfolder)
    tmax = 1e4 # replace with a calculation of tmax
    return tmax

In [7]:
%%time
if 'hasnull' not in dataset.columns:
    dataset['hasnull'] = dataset.apply(hasnull, axis=1)
    #dataset['tmax'] = dataset.apply(tmax, axis=1)
    # dataset['tmax'] = dataset.apply(lambda x:1e4, axis=1) # this version would just set tmax=1e4 for all of them

    #dataset.to_csv(trainingdatafolder+"trainingdata.csv", encoding='ascii')

CPU times: user 5.67 s, sys: 20.2 ms, total: 5.69 s
Wall time: 5.68 s


In [8]:
dataset

,Unnamed: 0,index,EMcrossnear,EMfracstdnear,EPstdnear,MMRstrengthnear,twoMMRstrengthnear,EMcrossfar,EMfracstdfar,EPstdfar,MMRstrengthfar,twoMMRstrengthfar,MEGNO,MEGNOstd,prelimStable,Stable,hasnull
0,0,0,0.221502,0.017511,0.002935,0.010371,0.010371,0.300893,0.085965,0.011956,0.000000,0.010371,1.942393,0.032212,True,False,0
1,1,1,0.190647,0.073477,0.011405,0.013957,0.013957,0.403777,0.005775,0.000461,0.000000,0.013957,22.918673,7.372319,True,False,0
2,2,2,0.125660,0.034270,0.001506,0.088234,0.088234,0.720557,0.016427,0.008207,0.000000,0.088234,11.884854,3.404385,True,False,0
3,3,3,0.216684,0.008561,0.002451,0.000000,0.000000,0.235460,0.003955,0.000146,0.005636,0.000000,2.000019,0.001839,True,True,0
4,4,4,0.024455,NaN,NaN,NaN,NaN,0.034058,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133538,3538,138538,0.082222,0.049940,0.016289,0.024447,0.024447,0.664222,0.031330,0.003745,0.000000,0.024447,1.998822,0.003453,True,False,0
133539,3539,138539,0.251338,0.016063,0.002925,0.000000,0.000000,0.272822,0.002159,0.000486,0.003788,0.000000,2.487767,0.188649,True,False,0
133540,3540,138540,0.131799,0.027892,0.001453,0.032653,0.032653,0.366664,0.067070,0.020456,0.000000,0.032653,7.134970,1.931917,True,False,0
133541,3541,138541,0.209454,0.090838,0.010214,0.037776,0.037776,0.395073,0.102280,0.038515,0.000000,0.037776,2.117164,0.088393,True,False,0


In [232]:
near = ['EMcrossnear', 'EMfracstdnear', 'EPstdnear', 'MMRstrengthnear','twoMMRstrengthnear']
far = ['EMcrossfar', 'EMfracstdfar', 'EPstdfar', 'MMRstrengthfar','twoMMRstrengthfar']
megno = ['MEGNO', 'MEGNOstd']
#what is being used to train
features = near + far + megno

pnear = ['EMcrossnear', 'EMfracstdnear', 'EPstdnear', 'MMRstrengthnear']
pfar = ['EMcrossfar', 'EMfracstdfar', 'EPstdfar', 'MMRstrengthfar']
pmegno = ['MEGNO', 'MEGNOstd']

pfeatures = pnear + pfar + pmegno

add columns for tmax and hasnull manually

Now we do the filtering manually. This is taking only systems with instability times > 1e4 AND no NaNs. Would adjust this for each case. 

We were worried that we were using filter=..., which was filtering out any rows that had any NaNs in them. We wanted to compare results when we don't include that filter, and only filtered for instability_time.

In [240]:
mask = (dataset['hasnull'] == 0 )
filtData = dataset[mask]

In [241]:
scrambled = filtData.sample(frac=1)

In [242]:
scrambled = scrambled.reset_index()

In [243]:
scrambled

,level_0,Unnamed: 0,index,EMcrossnear,EMfracstdnear,EPstdnear,MMRstrengthnear,twoMMRstrengthnear,EMcrossfar,EMfracstdfar,EPstdfar,MMRstrengthfar,twoMMRstrengthfar,MEGNO,MEGNOstd,prelimStable,Stable,hasnull
0,104456,4456,104456,0.185199,0.057962,0.002881,0.016910,0.016910,0.339532,0.014581,0.000694,0.000000,0.016910,1.817866,0.122473,True,False,0
1,131268,1268,136268,0.251526,0.021658,0.004632,0.000000,0.000000,0.620496,0.000828,0.002597,0.000000,0.000000,2.695655,0.245307,True,False,0
2,33695,3695,33695,0.044271,0.020562,0.001884,0.009357,0.009357,0.094144,0.029693,0.002126,0.004092,0.009357,2.408687,0.150927,True,False,0
3,67408,2408,67408,0.089061,0.050691,0.000076,0.012066,0.012066,0.143226,0.008068,0.000976,0.010308,0.012066,1.854385,0.074413,True,False,0
4,81947,1947,81947,0.069029,0.111012,0.002891,0.012180,0.012180,0.095413,0.040500,0.001429,0.008943,0.012180,51.803175,10.600187,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118949,37630,2630,37630,0.310737,0.034519,0.002335,0.000000,0.000000,0.593226,0.054703,0.036836,0.000000,0.000000,4.381696,0.512171,True,True,0
118950,124330,4330,124330,0.118009,0.073947,0.000220,0.024036,0.024036,0.897951,0.000367,0.000052,0.000000,0.024036,1.999741,0.000824,True,False,0
118951,38663,3663,38663,0.193345,0.060549,0.001441,0.019818,0.019818,0.353024,0.094407,0.024183,0.000000,0.019818,89.063506,14.086685,True,False,0
118952,97977,2977,97977,0.062857,0.043757,0.000940,0.028735,0.028735,0.455691,0.020736,0.000507,0.000000,0.028735,42.144196,15.304826,True,False,0


Now we do the train test split manually. We take 80\% of the rows for training, 20\% for testing (this is what we were doing before too)

In [244]:
#deciding training and not training, order is random
Nrows = int(0.8*scrambled.shape[0])
train = scrambled.iloc[:Nrows, :]
test = scrambled.iloc[Nrows:, :]
#test_tinst = tinst.iloc[Nrows:]

# dtrain = xgb.DMatrix(trainX, trainY)
# dtest = xgb.DMatrix(testX, testY)

In [245]:
#features var covers all features
train['Stable'].value_counts()[False]/train['Stable'].value_counts()[True]

2.3956467439785905

In [246]:
# trainFStab= train.loc[train['Stable']==False]
# trainTStab= train.loc[train['Stable']==True]

In [247]:
# BT = pd.concat([trainTStab,trainFStab.sample(frac=1)]).sample(frac=1)

In [179]:
realTest = test.loc[test['prelimStable']==True]

There are more higher EMcrossnear values in the wrong guesses

In [151]:
from sklearn.metrics import accuracy_score

In [248]:
twoModel = XGBClassifier(learning_rate = 0.03, 
                         max_depth = 30, 
                         subsample = 0.9,
                         min_child_weight = 5,
                         n_estimators = 300,
                         scale_pos_weight=2.4)

In [249]:
twoModel.fit(train[features], train["Stable"])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=30, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [250]:
oneModel = XGBClassifier(learning_rate = 0.03, 
                         max_depth = 20, 
                         subsample = 0.9,
                         min_child_weight = 5,
                         n_estimators = 200,
                         scale_pos_weight=2.4)

In [251]:
oneModel.fit(train[pfeatures], train["Stable"])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [252]:
resPred = twoModel.predict_proba(test[features])[:,1]
print (metrics.roc_auc_score(test['Stable'],resPred))
print( accuracy_score(twoModel.predict(realTest[features]), realTest["Stable"]))

0.955854531512888
0.9723424824513471


In [278]:
twoModel.save_model('SPOCKalt.bin')

In [276]:
new = XGBClassifier()
new.load_model('SPOCKalt.model')

In [262]:
train[features].iloc[0]

EMcrossnear           0.185199
EMfracstdnear         0.057962
EPstdnear             0.002881
MMRstrengthnear       0.016910
twoMMRstrengthnear    0.016910
EMcrossfar            0.339532
EMfracstdfar          0.014581
EPstdfar              0.000694
MMRstrengthfar        0.000000
twoMMRstrengthfar     0.016910
MEGNO                 1.817866
MEGNOstd              0.122473
Name: 0, dtype: float64

In [272]:
twoModel.predict(train[features].sample())

array([0])

In [253]:
resPred = oneModel.predict_proba(test[pfeatures])[:,1]
print (metrics.roc_auc_score(test['Stable'],resPred))
print(accuracy_score(oneModel.predict(realTest[pfeatures]), realTest["Stable"]))

0.9541572632612496
0.9593543777058552
